In [1]:
import fiona
import os
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
from ipywidgets import HTML
from IPython.display import clear_output
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Popup, Polyline, Polygon, LayerGroup, LayersControl,ScaleControl, FullScreenControl
from ipyleaflet import WidgetControl
from ipywidgets import interactive
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
#import matplotlib.patches as patches
#from matplotlib.patches import Polygon
import matplotlib
import matplotlib.colors
import contextily as cx
import shapely.geometry as sg
from rijksdriehoek import rijksdriehoek
from matplotlib.collections import PatchCollection
import ipympl
import warnings
import numpy as np
import ast

RD = rijksdriehoek.Rijksdriehoek()
plt.rcParams.update({'font.size': 24})

In [2]:
warnings.filterwarnings( "ignore")

In [3]:
def read_gpkg(filename):
    
    assert '.' in filename, f'Filename {filename} does not have an extension!'    
    if filename.split('.')[1] != 'gpkg':
        raise ValueError(f'Expected gpkg, got {filename}')
    if not os.path.isfile(f'Data/{filename}'):
        raise ValueError(f'File {filename} not found in Data directory!')
    
    AOI = {}
    with fiona.open(f"Data/{filename}") as layer:
        for feature in layer:     
            AOI[feature['properties']['ogc_fid']] = {
                "crd_dec": [feature['geometry']['coordinates'][ii] for ii in range(len(feature['geometry']['coordinates']))]
                }
            for key in ['h0', 's_h0', 'xI', 's_xI', 'xP', 's_xP', 'xE', 's_xE', 'tau', 's_tau',
                        'rho_xIh0', 'rho_xPh0', 'rho_xEh0', 'rho_tauh0',
                        'rho_xPxI', 'rho_xExI', 'rho_xItau', 
                        'rho_xPxE', 'rho_xPtau', 
                        'rho_xEtau', 'xI_frac']:
                AOI[feature['properties']['ogc_fid']][key] = feature['properties'][key]
    return AOI

def convert_wgs84_to_rd(list_of_coords):
    """
    Converts a list of coordinates in WGS84 to RD
    :param list_of_coords: list of (lon, lat) objects
    :return: list of (rd_x, rd_y) objects
    """
    rd_coords = []
    for coord in list_of_coords:
        RD.from_wgs(lon=coord[0], lat=coord[1])
        rd_coords.append([RD.rd_x, RD.rd_y])
    return rd_coords

def convert_rd_to_wgs84(list_of_coords):
    """
    Converts a list of coordinates in RD to WGS
    :param list_of_coords: list of (rdx, rdy) objects
    :return: list of (lon, lat) objects
    """
    wgs_coords = []
    for coord in list_of_coords:
        RD.rd_x = coord[0]
        RD.rd_y = coord[1]
        wgs_coords.append(RD.to_wgs())
    return wgs_coords

In [4]:
def read_aoi():
    filename = "Data/AoI_GreenHeart-polygon.shp"
    shape = fiona.open(filename)
    finished = False
    shape_iter = iter(shape)
    aoi_rd_poly = None
    while not finished:
        try:
            shp = next(shape_iter)
            polygons = [sg.Polygon(convert_wgs84_to_rd(shp["geometry"]["coordinates"][ii])) for ii in range(len(shp["geometry"]["coordinates"]))]
            if len(polygons) > 1 or aoi_rd_poly is not None:
                raise ValueError("Cannot handle AOI with more than 1 polygon!")
            aoi_rd_poly = polygons[0]
            if not aoi_rd_poly.is_valid:
                raise ValueError("Invalid polygon AOI!")
        except StopIteration:
            finished = True
    if aoi_rd_poly is None:
        raise ValueError(f"No valid AOI polygon found in {filename}!")
    return convert_rd_to_wgs84(np.array(aoi_rd_poly.exterior.coords.xy).T)
aoi_wgs_poly = read_aoi()

In [5]:
gpkg_data = read_gpkg('InSAR_ALS_GNSS_Grav_Leveling.gpkg')

In [6]:
output = widgets.Output()

In [7]:
#@widgets.interact_manual(plot=['xI', 's_xI', 'h0', 's_h0'])
def plot_overview(plot='xI'):
    with output:
        clear_output(wait=True)
        display(main_plot)
    if plot != 'Select':
        vdir = {'xI': [-10, 10],
                's_xI': [0, 10],
                'h0': [-8, 2],
                's_h0': [0, 10]}
                    
        with output:
            CarDB = basemap_to_tiles(basemaps.CartoDB.Positron)
            CarDB.base = True
            CarDB.name = 'Map (CartoDB)'
        
            img_nl = basemap_to_tiles(basemaps.nlmaps.luchtfoto)
            img_nl.base = True
            img_nl.name = 'Imagery (nlmaps)'
            
            m = Map(center=list(np.mean(np.array(aoi_wgs_poly), axis=0)), zoom=9, close_popup_on_click=False , layers = [img_nl,CarDB])

            AoI = LayerGroup(name = 'AoI')
            EUPs = LayerGroup(name = 'EUPs')
        
            m.add_layer(AoI)
            m.add_layer(EUPs)

            line = Polyline(locations = aoi_wgs_poly,
                        weight = 2,
                        color= '#000000', linestyle='--', fill=False) 

            # line.on_click(create_button_click_arc(j))
            AoI.add_layer(line)
            m.add_control(LayersControl())

            def convert_value_to_hex(value, vmin=-10, vmax=10):
                pct = min(max((value - vmin) / (vmax - vmin), 0), 1)
                clr = (int(255 * min(1, 2 - 2 * pct)), 
                       int(255 * (1 - 2 * abs(0.5 - pct))), 
                       int(255 * min(1, 2 * pct)))
                #clr = (int(255 * max(0, (1 - 2 * pct))), 0, (int(255 * max(2*pct - 1, 0))))
                hx = '#'
                for c in clr:
                    
                    hx += "{:0>2s}".format(hex(c)[2:])
                return hx
            
            for EUP in list(gpkg_data.keys())[:2000]:
                if plot in ['xI', 's_xI']:
                    val = gpkg_data[EUP][plot] * gpkg_data[EUP]['xI_frac'] * 365.2425 * 1000 # mm/y
                elif plot in ['h0']:
                    val = gpkg_data[EUP][plot] # m
                elif plot in ['s_h0']:
                    val = gpkg_data[EUP][plot] * 100 # cm
                else:
                    val = gpkg_data[EUP][plot] * 1000
                polygon = Polygon(
                    locations = convert_rd_to_wgs84(gpkg_data[EUP]['crd_dec'][0]),
                    weight=0,
                    fill_color=convert_value_to_hex(val, vdir[plot][0], vdir[plot][1]),
                    fill_opacity=1)
                EUPs.add_layer(polygon)



            display(m)

In [8]:
plot_drop = widgets.Dropdown(
        options=['Select', 'xI', 's_xI', 'h0', 's_h0'],
        value='Select',
        description='Parameter:',
        disabled=False,
    )
main_plot = interactive(plot_overview, plot=plot_drop)


In [9]:
%matplotlib ipympl
button = widgets.Button(description="Run Me!")


display(button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        display(main_plot)

button.on_click(on_button_clicked)

Button(description='Run Me!', style=ButtonStyle())

Output()